#Transferencia de aprendizagem e fine tunning

## importando bibliotecas e montando o drive

In [2]:
#importação das bibliotecas
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout

In [3]:
#montagem do drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#dataset
caminho = '/content/drive/MyDrive/estudo de visao computacional/Datasets/cat_dog_2.zip'
zipado = zipfile.ZipFile(caminho, mode='r')
zipado.extractall('./')
zipado.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/drive/MyDrive/estudo de visao computacional/Datasets/cat_dog_2.zip' mode='r'>>

In [6]:
gerador_treino = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.4,
                                    horizontal_flip=True,
                                    rotation_range=7,)
base_treino = gerador_treino.flow_from_directory('/content/cat_dog_2/training_set',
                                                 target_size=(128,128),
                                                 batch_size=10,
                                                 class_mode='binary',
                                                 shuffle=True)

Found 4000 images belonging to 2 classes.


In [7]:
gerador_teste = ImageDataGenerator(rescale=1./255)
base_teste = gerador_teste.flow_from_directory('/content/cat_dog_2/test_set',
                                               target_size=(128,128),
                                               batch_size=1,
                                               class_mode='binary',
                                               shuffle=False)

Found 1000 images belonging to 2 classes.


In [8]:
#chamando a rede pre treinada
modelo_base = tf.keras.applications.ResNet50(weights = 'imagenet', include_top=False,
                                             input_tensor = Input(shape=(128,128,3)))


94765736/94765736 [==============================] - 0s 0us/step


In [9]:
fine_tunning_at = 150
for layer in modelo_base.layers[:fine_tunning_at]:
  layer.trainable = False

In [10]:
modelo_base.output

<KerasTensor: shape=(None, 4, 4, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [11]:
modelo_head = modelo_base.output
modelo_head = GlobalAveragePooling2D()(modelo_head)
modelo_head = Dense(units=1025, activation='relu')(modelo_head)
modelo_head = Dropout(rate=0.2)(modelo_head)
modelo_head = Dense(units=1025, activation='relu')(modelo_head)
modelo_head = Dropout(rate=0.2)(modelo_head)
modelo_head = Dense(units=1, activation='sigmoid')(modelo_head)

In [12]:
rede = Model(inputs = modelo_base.inputs, outputs=modelo_head)

In [13]:
rede.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
historico = rede.fit(base_treino, epochs=70)

Epoch 1/70
400/400 [==============================] - 995s 2s/step - loss: 0.6751 - accuracy: 0.5863
Epoch 2/70
400/400 [==============================] - 987s 2s/step - loss: 0.6554 - accuracy: 0.6075
Epoch 3/70
400/400 [==============================] - 994s 2s/step - loss: 0.6384 - accuracy: 0.6395
Epoch 4/70
400/400 [==============================] - 1005s 3s/step - loss: 0.6241 - accuracy: 0.6597
Epoch 5/70
400/400 [==============================] - 1010s 3s/step - loss: 0.6193 - accuracy: 0.6572
Epoch 6/70
400/400 [==============================] - 1012s 3s/step - loss: 0.6052 - accuracy: 0.6710
Epoch 7/70
141/400 [=========>....................] - ETA: 10:52 - loss: 0.5948 - accuracy: 0.6865

KeyboardInterrupt: ignored